# Optimización de hiperparámetros del modelo aumentado


En este notebook se realizará la exploración de los hiperparámetros óptimos para entrenar un modelo basado en el modelo aumentado que se definio en notebook anterior. Esto se hará con el fin de mejorar el rendimient de este modelo.

Para ello se emplea la librería `keras_tuner`, que permite definir un espacio de soluciones para los hiperparámetros, que más tarde, mediante random search, se encontrarán distintas soluciones posibles óptimas.


In [1]:
import sys

sys.path.append("../")  # Add the parent directory to the Python path

In [3]:
import keras_tuner as kt
from src.models.tuneable_model import build_tunable_aumented_model
from src.data.data_loader import data_loader, data_loader_test

## Carga de los datos

Para este apartado se procede del mismo modo que se ha hecho hasta ahora


In [20]:
train_data, val_data = data_loader("../data/train", (150, 150), batch_size=128)

Found 12777 files belonging to 6 classes.
Using 10222 files for training.
Using 2555 files for validation.


## Random Search de las soluciones óptimas al problema


En primer lugar se instancia un objeto `tuner` que va a permitir realizar la optimización de las soluciones.

#### Parámetros de la Búsqueda Aleatoria

- `hypermodel`: Esta es la función que define el modelo que se va a optimizar. En este caso, estamos utilizando la función `build_tunable_aumented_model` para crear el modelo que queremos ajustar.

- `objective`: Especifica la métrica objetivo que se utilizará para evaluar los modelos generados durante la búsqueda. En este caso, estamos optimizando la "accuracy" (precisión).

- `max_trials`: Define el número máximo de modelos que se evaluarán de manera aleatoria. En este ejemplo, se han configurado un máximo de 50 modelos para ser evaluados.

- `directory`: Especifica el directorio en el que se guardarán los resultados de la búsqueda, incluidos los modelos y los registros. En este caso, los resultados se guardarán en "results_dir".

- `project_name`: Define el nombre del proyecto de búsqueda. Esto se utiliza para organizar los resultados de manera adecuada. En este caso, el proyecto se llama "keras_tuner_test".

- `overwrite`: Si se establece en `True`, sobrescribirá cualquier proyecto existente con el mismo nombre.


### Función `build_tunable_aumented_model`

Esta función es similar a las usadas para los casos del modelo simple y aumentado, con la excepción de que aquí se construye un modelo donde muchos de los hiperparámetros no están definidos por un valor, sino por un rango de valores. De este modo, no se puede usar este modelo para entrenar directamente, sino que primero hay que realizar la búsqueda de las distintas soluciones con las que cosntruir un modelo final.

Se ha definido un espacio de soluciones utilizando `hp.Choice`. Los hiperparámetros a optimizar y sus rangos son:

#### Funciones de Activación

- "relu"
- "sigmoid"
- "tanh"
- "selu"
- "elu"

#### Inicializadores de Pesos

- "glorot_uniform"
- "truncated_normal"
- "random_uniform"
- "ones"

#### Regularizadores

- "l1_l2"
- "l1"
- "l2"

#### Dropout

Los valores de Dropout variarán entre 0.1 y 0.6 en incrementos de 0.1.


In [18]:
tuner = kt.RandomSearch(
    hypermodel=build_tunable_aumented_model,
    objective="accuracy",
    max_trials=50,
    directory="results_dir",
    project_name="keras_tuner_test",
    overwrite=True,
)

In [19]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'selu', 'elu'], 'ordered': False}
kernel_initializer (Choice)
{'default': 'glorot_uniform', 'conditions': [], 'values': ['glorot_uniform', 'truncated_normal', 'random_uniform', 'ones'], 'ordered': False}
kernel_regularizer (Choice)
{'default': 'l1_l2', 'conditions': [], 'values': ['l1_l2', 'l1', 'l2'], 'ordered': False}
rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.6, 'step': 0.1, 'sampling': 'linear'}


#### Iniciar la Búsqueda

Una vez que se ha configurado la búsqueda aleatoria de hiperparámetros con los parámetros mencionados anteriormente, se inicia la búsqueda utilizando el siguiente código:


In [21]:
tuner.search(
    train_data,
    epochs=2,
)

Trial 50 Complete [00h 01m 53s]
accuracy: 0.545881450176239

Best accuracy So Far: 0.6016435027122498
Total elapsed time: 01h 37m 17s


### Elección del modelo óptimo

En este punto se exploran las soluciones y se elige como `best_tuned` el que haya obtenido mejor accuracy en las epochs en las que se han probado


In [23]:
models = tuner.get_best_models(num_models=1)
best_tuned = models[0]
best_tuned.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                         

### Guardado de la solución óptima

A continuación se guarda el mejor modelo obtenido junto al resto de modelos entrenados, esto se hace con el fin de poder utilizarlo posteriormente para inferencia o para continuar su entrenamiento.


In [26]:
MODEL_SAVE_PATH = "../models/aumented_tuned"
best_tuned.save(MODEL_SAVE_PATH)

INFO:tensorflow:Assets written to: ../models/aumented_tuned\assets


INFO:tensorflow:Assets written to: ../models/aumented_tuned\assets


Llegados a este punto se ha encontrado un modelo que es el que se mejores hiperparámetros tiene para converger a una solución óptima. Se abrirá este modelo en un notebook posterior para continuar su entrenamiento y analizar sus resultados sobre los distintos sets.
